In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from card_detector import CardDetector, fit
from dataloader_utils import MTGCardsDataset, get_transform_pipe
from config import *

In [2]:
torch.cuda.empty_cache()

In [3]:
anchor_boxes = torch.Tensor([[198.27963804, 206.74086672],
       [129.59395666, 161.90171490],
       [161.65437828, 232.34624509]
]) # Anchor boxes acquired from k-means clustering of the dataset

model = CardDetector(img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
                      anchor_boxes=anchor_boxes,
                        num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
                          loss_fn=nn.L1Loss(reduction="sum")
                          )

dummy_image = torch.randn(1, 3, CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]) # create a dummy image to pass to the model and get feature map size
feature_map_shape = model.feature_extractor(dummy_image).shape
feature_map_dims = (feature_map_shape[-2], feature_map_shape[-1])


transform_pipe = get_transform_pipe(img_w=CONFIG["dataset"]["img_w"], img_h=CONFIG["dataset"]["img_h"])

train_dataset = MTGCardsDataset(
  annotations_file=CONFIG["dataset"]["annotations_file"],
  img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
  anchor_boxes=model.anchor_boxes,
  feature_map_dims=feature_map_dims,
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  num_max_boxes=1,
  transform=transform_pipe,
  limit=10_000
)
train_dataloader = DataLoader(train_dataset, batch_size=CONFIG["dataloader"]["batch_size"])

val_dataset = MTGCardsDataset(
  annotations_file=CONFIG["dataset"]["annotations_file"],
  img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
  anchor_boxes=model.anchor_boxes,
  feature_map_dims=feature_map_dims,
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  num_max_boxes=1,
  transform=transform_pipe,
  limit=1000
)
val_dataloader = DataLoader(val_dataset, batch_size=CONFIG["dataloader"]["batch_size"]) 

c:\Users\vikio\anaconda3\envs\nsiete_pytorch_project\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vikio\anaconda3\envs\nsiete_pytorch_project\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
opt = torch.optim.Adam(params=model.parameters(), lr=CONFIG["optimizer"]["lr"])
fit(model=model, num_epochs=5, optimizer=opt, train_dataloader= train_dataloader, val_dataloader= val_dataloader, device= ("cuda" if torch.cuda.is_available() else "cpu"))

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0



c:\Users\vikio\anaconda3\envs\nsiete_pytorch_project\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Looked at 0 Batches	---	0/10000 Samples
Looked at 100 Batches	---	3200/10000 Samples
Looked at 200 Batches	---	6400/10000 Samples
Looked at 300 Batches	---	9600/10000 Samples
Looked at 312 Batches	---	10000/10000 Samples
Train loss: 0.08157331809496728 | Val Loss: 0.012037569656968117 | Val Accuracy: 0.0
Epoch: 1

Looked at 0 Batches	---	0/10000 Samples
Looked at 100 Batches	---	3200/10000 Samples


KeyboardInterrupt: 

In [ ]:
from torchvision.io import read_image, ImageReadMode
from torchvision.utils import draw_bounding_boxes
import torchvision.transforms.functional as fn

test_image = read_image("./data/augmented_1/991_3.png", mode=ImageReadMode.RGB)
test_transform_pipe = get_transform_pipe(640, 640)
model_input = test_transform_pipe(test_image)

model = model.to("cpu")
pred_box = model.predict(model_input)#, ground_truth=feature_label.unsqueeze(0))
pred_box = pred_box.squeeze(0)
print(pred_box)

#test_image = fn.convert_image_dtype(image=test_image, dtype=torch.uint8)
bbox_img_tensor = draw_bounding_boxes(image=test_image, boxes=pred_box, width=3, colors=(255,255,0))

fn.to_pil_image(bbox_img_tensor)